## Save masks
Load the numpy mask files and save them to zarr

In [ ]:
import dask.array as da
import numpy as np
from liom_toolkit.utils import save_label_to_zarr, generate_label_color_dict_mask, load_zarr
from ome_zarr.dask_utils import resize
from tqdm.auto import tqdm

In [ ]:
mask_files = []
zarr_files = []
# Colour dict for setting the label colour in napari
colour_dict = generate_label_color_dict_mask()
scales = (6.5, 6.5, 6.5)
# The scale at which the mask is generated
resolution_level = 2

In [ ]:
for mask_file, zarr_file in tqdm(zip(mask_files, zarr_files)):
    mask = da.from_array(np.load(mask_file))
    # Switch x and z axis for zarr
    mask = da.transpose(mask, (2, 1, 0))
    # Get the image size
    nodes = load_zarr(zarr_file)
    image_node = nodes[0]
    image_shape = image_node.data[0].shape
    if len(image_shape) == 4:
        image_shape = image_shape[1:]
    mask = resize(mask, image_shape)

    save_label_to_zarr(mask, zarr_file, colour_dict, name="mask", scales=scales)